WISO100303 / Johannes Schmidt & Peter Regner

# **An introduction to scientific programming**

<br> <br> <br> <br><br> <br> <br> <br>

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
FIGSIZE = (12, 12)  # used later for changing figure size

In [ ]:
# donwload the data
def download_attached_files():
    import urllib
    import os.path
    fnames = {
              'vienna-map.png': 'https://files.boku.ac.at/filr/public-link/file-download/0d7483c99572915f0195892102336dfd/18659/-4041302498646712195/vienna-map.png',
              'public-toilets.csv': 'https://files.boku.ac.at/filr/public-link/file-download/0d7483c99572915f01958920fab66dd5/18658/-1977532774310129682/public-toilets.csv',
    }
    for fname, url in fnames.items():
        if not os.path.exists(fname):
            urllib.request.urlretrieve(url, filename=fname)

download_attached_files()

# Recap: Numpy arrays

Last time we used `np.array()` to create arrays. There are some other helpful functions to create arrays:

In [ ]:
many_ones = np.ones([4,4])
many_ones

In [ ]:
many_ones[0]

In [ ]:
many_ones[0, 0]

Note that indexing can be used to assign values too:

In [ ]:
many_ones[-1, -1] = 0

In [ ]:
many_ones

In [ ]:
many_ones[-1, :] = 0

In [ ]:
many_ones

Today we will focus on more methods to do indexing and slicing of arrays.

# Let's do something with real data

Public toilets in Vienna:
https://www.data.gv.at/datasets/b25202cf-c129-34ca-bdce-d843dd638eaf

Note: we use the order (longitude, latitude) here. This is the order used in shape files and necessary for plotting too.

In [ ]:
import urllib
import os.path
from pathlib import Path
import pandas as pd

def read_public_toilets():
    """Download CSV with geocordinates of public toilets in Vienna, parse it and return a numpy
    array of shape (N,2), where each point is (longitude_x, latitude_y)."""
    fname = 'public-toilets.csv'
    if not os.path.exists(fname):
        URI = ('https://data.wien.gv.at/daten/geo?'
               'service=WFS&request=GetFeature&version=1.1.0&'
               'typeName=ogdwien:WCANLAGEOGD&srsName=EPSG:4326&outputFormat=csv')
        urllib.request.urlretrieve(URI, filename=fname)
        
    d = pd.read_csv(fname)
    return d.SHAPE.str.extract(r'POINT \((\d+\.\d+) (\d+\.\d+)\)').astype(float).values

In [ ]:
# note that the CSV file will not be re-downloaded from the official website unless it is deleted
# in the Datalore Notebook files without running download_attached_files() at the beginning of the notebook
public_toilets = read_public_toilets()

In [ ]:
public_toilets.shape

In [ ]:
public_toilets[:5]

In [ ]:
stephansplatz = np.array([16.372223, 48.208432])

In [ ]:
# Length of one degree longitude/latitude on Stephansplatz approximately:
LON_TO_KM = 74.1
LAT_TO_KM = 111.19

Doing calculations with longitude/latitude is difficult. Let's project the points to Cartesian coordinates in kilometers with origin at the Stephansplatz. Of course this it's not very accurate to project longitude/latitude that way, but for a small area like Vienna it should be good enough:

In [ ]:
def to_km(locations):
    return (locations - stephansplatz) * np.array([LON_TO_KM, LAT_TO_KM])

In [ ]:
public_toilets_km = to_km(public_toilets)

In [ ]:
public_toilets_km[:5]

**This is not a good way how to work with maps for larger areas, even if it works reasonably well on small scale. This is just an example of how to use Numpy!**

# Numpy broadcasting rules

<!-- broken in Datalore, see cell below
<img src="numpy_broadcasting.png" width="800">
-->

<small>
Source: [scipy lecture notes](http://scipy-lectures.org/intro/numpy/operations.html#broadcasting) (CC-BY 4.0)
</small>

- operations with Numpy are (mostly) elementwise.
- if the shape does not match, the smaller array is duplicated along missing axis

![Numpy broadcasting rules](images/numpy_broadcasting.png)

Examples:

    A      (2d array):  5 x 4
    B      (1d array):      1
    Result (2d array):  5 x 4

    A      (2d array):  5 x 4
    B      (1d array):      4
    Result (2d array):  5 x 4

    A      (3d array):  15 x 3 x 5
    B      (3d array):  15 x 1 x 5
    Result (3d array):  15 x 3 x 5

Full documentation:
https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

**How does that work in detail?**

NumPy broadcasting allows arrays of different shapes to undergo element-wise operations.

Sizes of the dimensions are compared, starting with the trailing (i.e. rightmost) one:
- if the sizes match, there is nothing to do
- if the size equals 1 or if there is no size due to a lower number of dimensions, the smaller array is copied along the dimensions to match the shape of the larger array.
- if the sizes do not match and both are >1, an error is raised: "ValueError: operands could not be broadcast together"

**Hint:** With more than two dimensions things get a bit confusing. Consider using the library `xarray` if you need that often!

Ok, so can we plot our data set to get a better overview?

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.plot(public_toilets_km.T[0], public_toilets_km.T[1], 'ko', markersize=3, label='Public toilet')
plt.plot([0], [0], 'P', label='Stephansplatz', color='chocolate')

plt.gca().set_aspect('equal')
plt.legend()
plt.title('Map of public toilets in Vienna')
plt.xlabel('Longitude [km]')
plt.ylabel('Latitude[km]')
plt.grid();

# Reading and plotting images

Let's add a simple (inaccurate) map of Vienna (adapted version from [this one](https://commons.wikimedia.org/wiki/Category:Maps_of_Vienna#/media/File:Gemeindebezirke_Wiens.svg), license: public domain). See also [interactive map of public toilets](https://m.wien.gv.at/stadtplan/#base=karte&zoom=12&lat=48.2023&lon=16.4142&layer=wc).

In [ ]:
vienna = plt.imread('vienna-map.png')

In [ ]:
vienna.shape

The third dimension here is the color space. If you have never seen RGB values before, you can [google for "color picker"](https://www.google.com/search?q=color+picker) to see a demonstration.

In [ ]:
# distance from Stephansplatz to borders of the PNG file in km
left = -13.682179147809752
right = 16.639373238835283
bottom = -9.81358722568525
top = 12.989326274442831
extent = left, right, bottom, top

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.plot(public_toilets_km.T[0], public_toilets_km.T[1], 'ko', markersize=3, label='Public toilet')
plt.plot([0], [0], 'P', label='Stephansplatz', color='chocolate')
plt.gca().set_aspect('equal')
plt.legend()
plt.title('Map of public toilets in Vienna')
plt.xlabel('Longitude [km]')
plt.ylabel('Latitude[km]')
plt.grid()

plt.imshow(vienna, extent=extent);

There are [better ways to plot geographic maps](https://matplotlib.org/basemap/users/examples.html). This is just a simple example to play with Numpy.

# Distances between locations

In [ ]:
def distance(point1, point2):
    """Calculate Euclidean distance between two points. Points are passed
    as lists or arrays of length 2. Numpy arrays of many dimensions are 
    supported: axis=0 must be the dimension for x/y coordinates."""
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [ ]:
a = np.array([1, 1])
b = np.array([0, 0])

In [ ]:
distance(a, b)

In [ ]:
you_are_here =  np.array([16.3365605,48.2364388])

Unfortunately the array has the wrong shape, the first _axis_ must be the one for x/y coordinates:

In [ ]:
public_toilets_km.shape

Transposing the array solves this problem:

In [ ]:
public_toilets_km.T.shape

Why is this necessary? We can look up how `distance()`:

In [ ]:
distance?

In [ ]:
distance??

In [ ]:
public_toilets_km[:4]

In [ ]:
public_toilets_km[:4].T

In [ ]:
public_toilets_km[0]

In [ ]:
public_toilets_km.T[0]

In [ ]:
distances_to_me = distance(public_toilets_km.T, to_km(you_are_here))

In [ ]:
distances_to_me[:5]

There is a toilet very close, less than 500m!

In [ ]:
distances_to_me.min()

Often there is another way to call Numpy functions:

In [ ]:
np.min(distances_to_me)

<small>Note: In Numpy, usually there is no difference between these two ways of calling either the function `np.min(distances_to_me)` or the method `distances_to_me.min()`. A convention says methods should be used when the object is modified and the function if a new object is returned. But it this convention is not very widesprad, it is not necessary to stick to it.

But where is the toilet?

In [ ]:
closest_idx = distances_to_me.argmin()
closest_idx

In [ ]:
public_toilets[closest_idx]

In [ ]:
def google_maps_link(location):
    """Return link to turbine in Google maps.
    
    See documentation:
    https://developers.google.com/maps/documentation/urls/guide
    https://stackoverflow.com/questions/47038116/google-maps-url-with-pushpin-and-satellite-basemap

    """
    xlong, ylat = location
    
    # alternative API which does not allow marker
    # f"https://www.google.com/maps/@?api=1&map_action=map&center={ylat},{xlong}&basemap=satellite"
    
    # alternative API which does not allow sattelite
    # f"https://www.google.com/maps/search/?api=1&query={ylat},{xlong}"
    
    # zoom level z=xxx seems to be broken somehow (?)
    return f"http://maps.google.com/maps?q=loc:{ylat}+{xlong}&z=13"

In [ ]:
print(google_maps_link(public_toilets[closest_idx]))

In [ ]:
print(google_maps_link(you_are_here))

In [ ]:
def osm_location_link(location, zoom_level):
    """Return link to location in openstreetmaps."""
    xlong, ylat = location
    return f"https://www.openstreetmap.org/?mlat={ylat}&mlon={xlong}#map={zoom_level}/{ylat}/{xlong}"

In [ ]:
print(osm_location_link(public_toilets[closest_idx], 15))

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.plot(public_toilets_km.T[0], public_toilets_km.T[1], 'ko', markersize=3, label='Public toilet');
plt.plot([0], [0], 'P', label='Stephansplatz', color='chocolate')
plt.gca().set_aspect('equal')
plt.title('Map of public toilets in Vienna')
plt.xlabel('Longitude [km]')
plt.ylabel('Latitude[km]')
plt.grid()

plt.imshow(vienna, extent=extent)

plt.plot(public_toilets_km.T[0][closest_idx], public_toilets_km.T[1][closest_idx],
         'mv', label='Closest toilet')
plt.plot(*to_km(you_are_here), '*g', label='You are here')

plt.legend();

## Exercise 1 - Find the hottest year

Find the hottest year!

<small>Data source: https://www.wien.gv.at/statistik/wetter/</small>

In [ ]:
years = np.array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
                  1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
                  1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
                  2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013])

mean_temperature_celsius = np.array([10.2, 8.6, 8.7, 9.1, 8.6, 10.1, 10.2, 9.7, 9.2, 9.2, 9.8, 9.3, 9.6,
                                     10.2, 10.1, 9.6, 10.1, 9.1, 9.6, 8.7, 10.1, 10.0, 10.8, 9.4, 9.0, 9.6,
                                     9.3, 10.4, 10.7, 10.9, 9.7, 11.1, 10.8, 11.8, 10.4, 8.9, 10.0, 10.8,
                                     10.7, 11.7, 10.6, 11.3, 11.0, 10.4, 10.2, 10.7, 11.7, 11.4, 11.0, 9.9,
                                     11.1, 11.3, 10.9])

In [ ]:
# # # # # YOUR SOLUTION GOES HERE # # # # #

<div style="color:#555;border-top:1px solid #999;text-align:right;padding:4px;">End of exercise</div>

# How many close toilets are there?

In [ ]:
distances_to_me < 1.

In [ ]:
np.sum(distances_to_me < 1.)

## Exercise 2 - How many hot years?

Count how many hot years there are with more than 11°C in average (use the data from exercise 1).

In [ ]:
# # # # # YOUR SOLUTION GOES HERE # # # # #

<div style="color:#555;border-top:1px solid #999;text-align:right;padding:4px;">End of exercise</div>

# Indexing arrays in multiple dimensions

We have seen already that indexing works with square brackets:

In [ ]:
public_toilets[0]

Numpy supports indexing also in more than one dimension:

In [ ]:
public_toilets[0, 1]

The slice operations with `:` work also in multiple dimensions:

In [ ]:
public_toilets[:3, 0:1]

In [ ]:
some_toilets = public_toilets[:4, :]
some_toilets

Note that passing `:` as second index, is equivalent to omitting the second index for a 2D Numpy array:

In [ ]:
public_toilets[:4]

One can also pick more than one element at once by providing a list of indices:

In [ ]:
public_toilets[[0,3,5]]

The later is also called [fancy indexing](https://scipy-lectures.org/intro/numpy/array_object.html#fancy-indexing).

# Filtering: Fancy indexing using boolean arrays

Recall: we calculated the distance from our current position to each toilet and then created a boolean array which is `True` for each toilet closer than 1km:

In [ ]:
distances_to_me

In [ ]:
distances_to_me < 1.

In [ ]:
is_close_to_me = distances_to_me < 1.

This boolean array can be used to select entries of the array:

In [ ]:
close_toilets = public_toilets_km[is_close_to_me]
close_toilets

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.plot(*public_toilets_km.T, 'ko', markersize=3, label='Public toilet')
plt.plot(*close_toilets.T, 'rx', markersize=5, label='Toilet close to me')

plt.plot([0], [0], 'P', label='Stephansplatz', color='chocolate')
plt.gca().set_aspect('equal')
plt.legend()
plt.title('Map of public toilets in Vienna')
plt.xlabel('Longitude [km]')
plt.ylabel('Latitude[km]')
plt.grid()

plt.imshow(vienna, extent=extent);

## Exercise 3 - Which are the hot years?

Find the years with more than 11°C average temperature (use the data from exercise 1).

Bonus: find the coldest years, i.e. years with average temperature less than 9°C.

In [ ]:
# # # # # YOUR SOLUTION GOES HERE # # # # #

<div style="color:#555;border-top:1px solid #999;text-align:right;padding:4px;">End of exercise</div>

## Exercise 4 - Temperature via indices

Use fancy index techniques to print the years and temperatures from exercise 2 for the given indices.

In [ ]:
indices = np.array([0, -2, -1])

In [ ]:
# # # # # YOUR SOLUTION GOES HERE # # # # #

<div style="color:#555;border-top:1px solid #999;text-align:right;padding:4px;">End of exercise</div>

# bonus part

# Adding a new axis

One can even create new dimensions:

In [ ]:
some_toilets.shape

In [ ]:
some_toilets[:, :, np.newaxis].shape

In [ ]:
some_toilets[:, :, np.newaxis]

# More advanced broadcasting

In [ ]:
a = np.arange(5)
a

We can use `np.ones()` to create a column vector, i.e. an array of shape `(n, 1)`:

In [ ]:
b = np.ones((5, 1)) * 2
b

In [ ]:
b.shape

In [ ]:
a * b

# Fancy indexing with toilets 

Let's create a discrete grid covering Vienna. We will use a cartesian coordinate system in kilometers with origin at the Stephansplatz:

In [ ]:
x = np.linspace(left, right, num=150)
y = np.linspace(bottom, top, num=150)

In [ ]:
x[:5]

`xx` is an 2D array where each element contains the x coordinate of corresponding grid point, `yy` contains the y coordinates:

In [ ]:
xx, yy = np.meshgrid(x, y)

In [ ]:
xx.shape

In [ ]:
yy.shape

Note that all values in each column are the same, because the x coordinate does not change in that direction:

In [ ]:
xx[:3, :3]

To allow broadcasting between `public_toilets_km` and `xx` and `yy` new dimensions of size 1 are needed:

In [ ]:
public_toilets_km.shape

In [ ]:
public_toilets_km_grid = public_toilets_km.T[:, :, np.newaxis, np.newaxis]
public_toilets_km_grid.shape

In [ ]:
grid = np.array([xx, yy])[:, np.newaxis, :, :]
grid.shape

For each grid pixel there is a distance calculation to each toilet:

In [ ]:
distances = distance(grid, public_toilets_km_grid)
distances.shape

Let's find the closest toilet for each grid pixel!

In [ ]:
min_distance = distances.min(axis=0)

In [ ]:
min_distance.shape

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.plot(public_toilets_km.T[0], public_toilets_km.T[1], 'ko', markersize=3, label='Public toilet')
plt.plot([0], [0], 'P', label='Stephansplatz', color='chocolate')
plt.gca().set_aspect('equal')
plt.legend()
plt.title('Map of public toilets in Vienna')
plt.xlabel('Longitude [km]')
plt.ylabel('Latitude[km]')
plt.grid()

plt.imshow(min_distance, extent=extent, origin='lower',
           cmap='viridis_r',
           norm=matplotlib.colors.PowerNorm(0.35)
)
plt.colorbar(label='Distance to closest toilet [km]')
plt.imshow(vienna, extent=extent);

## Exercise 5 - Plot a two dimensional Gaussian

Plot the [two dimensional Gaussian function](https://en.wikipedia.org/wiki/Gaussian_function#Two-dimensional_Gaussian_function):
$$
    g(x,y) = \exp\left(- \left(\frac{x^2}{2\sigma_X^2} + \frac{y^2}{2\sigma_Y^2} \right)\right)
$$

Plot the function in the range $x=-10,\ldots,10$ and $y=-10,\ldots,10$ and use $\sigma_X=10$ and $\sigma_Y=7$.

In [ ]:
# # # # # YOUR SOLUTION GOES HERE # # # # #

<div style="color:#555;border-top:1px solid #999;text-align:right;padding:4px;">End of exercise</div>

## Exercise 6 - Find optimum in plot

Alice wants to produce toilet paper. She can choose production costs per pallet freely between 5 and 40 EUR and the price between 15 and 70 EUR. Higher costs will lead to more products sold (due to higher quality), a higher price will lead to less products sold.

Assume the number of sold items is given by:
$$
    n = 1e5 \cdot \frac{\log(\textrm{costs})}{\textrm{price}^{0.6}}
$$

The profit is given by:

$$
    \textrm{profit} = n \cdot (\textrm{price} - \textrm{costs})
$$

Set discrete values for `costs` and `price` using `np.linspace()`. Then convert costs to a column vector using 

```costs = costs[:, np.newaxis]```
    
(check the shape before and after!). Then calculate the profit.

Plot the profit using `plt.imshow()` and analyze the parameters for the maximum profit!

Use following additional parameters for `plt.imshow()`:

`
extent=[price.min(), price.max(),
       costs.min(), costs.max()],
origin='lower'
`

Use the command `plt.colorbar()` to add a color legend.

In [ ]:
# # # # # YOUR SOLUTION GOES HERE # # # # #

<div style="color:#555;border-top:1px solid #999;text-align:right;padding:4px;">End of exercise</div>

# Worst spot for urgent needs

Which are the worst spots for urgent needs, where you should avoid drinking beer outside?

<small>(At the moment, drinking beer outside is still better than inside with someone you are not living together anyway!)</small>

In [ ]:
from scipy.ndimage import maximum_filter

A maximum filter sets each element to the maximum value of its surroundings:

In [ ]:
maximum_filter([1, 1, 1, 5, 1, 1, 1], size=3)

This is a very neat trick to find local maxima in a discrete data set:

In [ ]:
is_peak = maximum_filter(min_distance, 10) == min_distance

`is_peak` is True, if all the element is at least as large as all others in the surrounding. Such a point is a local maximum.

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.plot(public_toilets_km.T[0], public_toilets_km.T[1], 'ko', markersize=3, label='Public toilet')
plt.plot([0], [0], 'P', label='Stephansplatz', color='chocolate')
plt.gca().set_aspect('equal')

plt.plot(xx[is_peak], yy[is_peak], 'rx', label='Bad spot for urgent needs')

plt.legend()
plt.title('Map of public toilets in Vienna')
plt.xlabel('Longitude [km]')
plt.ylabel('Latitude[km]')
plt.grid()

plt.imshow(min_distance, extent=extent, origin='lower',
           cmap='viridis_r',
           norm=matplotlib.colors.PowerNorm(0.35)
)

plt.imshow(vienna, extent=extent);

# Reshaping arrays

We got a discrete grid of points covering Vienna, each contains the distance to the closest toilet:

In [ ]:
min_distance.shape

In [ ]:
min_distance.flatten()

In [ ]:
min_distance.flatten().shape

# Histogram

How likely is it to be close to a public toilet?

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.hist(min_distance.flatten(), bins=100, density=True)
plt.xlabel('Distance to the next toilet [km]')
plt.ylabel('Probability');

But this uses all pixels in the grid... Wouldn't it be more interesting to look only at the center?

Let's filter the grid for all points closer to the center than 8km:

In [ ]:
grid[:, 0].shape

Remember that the center is at `[0, 0]`:

In [ ]:
is_central = distance(grid[:, 0], np.zeros((2, 1, 1))) < 8

In [ ]:
plt.figure(figsize=FIGSIZE)

plt.plot(public_toilets_km.T[0], public_toilets_km.T[1], 'ko', markersize=3, label='Public toilet')
plt.plot([0], [0], 'P', label='Stephansplatz', color='chocolate')
plt.gca().set_aspect('equal')

plt.legend()
plt.title('Map of public toilets in Vienna')
plt.xlabel('Longitude [km]')
plt.ylabel('Latitude[km]')
plt.grid()

plt.imshow(min_distance, extent=extent, origin='lower',
           cmap='viridis_r',
           norm=matplotlib.colors.PowerNorm(0.35)
)

plt.colorbar(label='Distance to closest toilet [km]')

plt.imshow(is_central,
           extent=extent,
           origin='lower',
           cmap='gray',
           alpha=0.2)

plt.imshow(vienna, extent=extent);

In [ ]:
plt.hist(min_distance[is_central], bins=100, density=True)
plt.xlabel('Distance to the next toilet [km]')
plt.ylabel('Probability');

Note that the result is always flat when using a boolean array as index!

# Interpolation

Let's assume we got `min_distance` as dataset from somewhere else and we cannot compute the distance by calling `distance()`. How could we get a good value?

In [ ]:
# x and y have been overwritten in the exercise before, so let's re-create:
x = np.linspace(left, right, num=150)
y = np.linspace(bottom, top, num=150)

In [ ]:
you_are_here_km = to_km(you_are_here)

Unfortunately the point does not lie on the grid:

In [ ]:
you_are_here_km[0] in x

In [ ]:
from scipy.interpolate import interp2d

In [ ]:
x.shape

In [ ]:
min_distance.shape

In [ ]:
distance_interpolated = interp2d(x, y, min_distance)

In [ ]:
distance_interpolated(you_are_here_km[0], you_are_here_km[1])

In [ ]:
distance(you_are_here_km, public_toilets_km.T).min()